# Question 1

In [41]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np
import autograd as ag

In [42]:
price_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\price_response.csv")

In [43]:
# Extract the "Intercept", "Capacity", and "Sensitivity" column
intercept_values = price_df['Intercept'].values.reshape(3, -1)
product_capacity = price_df['Capacity'].values.reshape(3, -1)
slope = price_df['Sensitivity'].values.reshape(3, -1)

### Part (a)

In [44]:
from scipy.optimize import minimize

# Objective function (negated for maximization)
def objective(p):
    p1, p2 = p
    return -1 * (p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2))

# Constraint functions
def constraint1(p):
    return p[1] - p[0]

def constraint2(p):
    return intercept_values[0,0] + slope[0,0] * p[0]

def constraint3(p):
    return intercept_values[0,1] + slope[0,1] * p[1]

# KKT conditions using only constraint1
def kkt_conditions(p, lagrange_multipliers):
    grad_objective = [-(intercept_values[0,0] + 2 * slope[0,0] * p[0]), -(intercept_values[0,1] + 2 * slope[0,1] * p[1])]
    grad_constraint1 = [-1, 1]
    
    # Stationarity condition
    stationarity = [grad_objective[i] + lagrange_multipliers[0] * grad_constraint1[i] for i in range(len(p))]
    
    # Complementary slackness conditions
    complementary_slackness = lagrange_multipliers[0] * constraint1(p)
    
    # Feasibility conditions
    feasibility = constraint1(p)
    
    return stationarity + [complementary_slackness] + [feasibility]

# Initial guess for Lagrange multipliers
initial_lagrange_multipliers = [1.0]

# Bounds for Lagrange multipliers
bounds_lagrange_multipliers = [(0, None)] * len(initial_lagrange_multipliers)

# Initial guess
initial_guess = [0, 0]

# Bounds for variables
bounds = [(0, None), (0, None)]  # P1 and P2 are non-negative

# Solve using minimize with original objective and constraints
result = minimize(objective, initial_guess, bounds=bounds, constraints=[{'type': 'ineq', 'fun': constraint1},
                                                                       {'type': 'ineq', 'fun': constraint2},
                                                                       {'type': 'ineq', 'fun': constraint3}])

# Extract solution
p_solution = result.x
maximized_profit = -1 * result.fun  # Convert back to positive for interpretation

# Now that p_solution is defined, you can use it in the minimize function for Lagrange multipliers

result = minimize(lambda lagrange_multipliers: sum([val**2 for val in kkt_conditions(p_solution, lagrange_multipliers)]),
                  initial_lagrange_multipliers,
                  bounds=bounds_lagrange_multipliers)

# Extract Lagrange multipliers solution
lagrange_multipliers_solution = result.x

print("Optimal values:")
print("P1:", p_solution[0])
print("P2:", p_solution[1])
print("Maximized Revenue:", maximized_profit)
print("Lagrange Multipliers:", lagrange_multipliers_solution)

Optimal values:
P1: 383.8504532037101
P2: 2296.4968098122035
Maximized Revenue: 50154983.343558624
Lagrange Multipliers: [5.72813245e-08]


### (b)

In [45]:
# Objective function
def objective_function(p1, p2):
    return p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2)

# Projected Gradient Descent with Gurobi
def projected_gradient_descent_with_gurobi(learning_rate, threshold):
    # Initialize Gurobi model
    model = gb.Model("projected_gradient_descent")
    
    # Decision variables
    p1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p1")
    p2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p2")
    
    # Objective
    obj = p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2)
    model.setObjective(obj, GRB.MAXIMIZE)
    
    # Optimization loop
    prev_obj = float('inf')
    i = 0
    
    while True:
        # Price Constraint  
        model.addConstr(p1 <= p2, f"Price Constraint {i}")

        model.addConstr(intercept_values[0,0] + slope[0,0]*p1 >= 0, "Demand Definition Product Line 1 Basic")
        model.addConstr(intercept_values[0,1] + slope[0,1]*p2 >= 0, "Demand Definition Product Line 1 Advance")
        
        # Optimize model
        model.optimize()
        
        # Get current solution
        current_p1 = p1.X
        current_p2 = p2.X
        
        # Compute objective function
        current_obj = objective_function(current_p1, current_p2)
        
        # Check convergence
        if (current_obj - prev_obj) < threshold:
            break
        
        prev_obj = current_obj
        
        # Compute gradient
        df_dx = intercept_values[0,0] + 2 * slope[0,0] * current_p1
        df_dy = intercept_values[0,1] + 2 * slope[0,1] * current_p2
        
        # Update parameters
        current_p1 -= learning_rate * df_dx
        current_p2 -= learning_rate * df_dy
        
        # Set new starting point
        p1.Start = current_p1
        p2.Start = current_p2
        
        # Increment iteration counter
        i += 1
    
    return current_p1, current_p2

# Hyperparameters
learning_rate = 0.001
threshold = 1e-6

# Run projected gradient descent with Gurobi
final_x, final_y = projected_gradient_descent_with_gurobi(learning_rate, threshold)
print(f"Final solution: x = {final_x}, y = {final_y}, Objective = {round(objective_function(final_x, final_y),10)}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x993e9687
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 4e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Tim

### (c)

In [46]:
# Create the optimization model
part_c_model = gb.Model("Question 1c): TechEssentials Certain Product Line")

p = part_c_model.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name="Price")

#Objective Function
part_c_model.setObjective(gb.quicksum(p[i,n]*(intercept_values[i,n] + slope[i,n]*p[i,n]) for i in range(3) for n in range(3)), GRB.MAXIMIZE)

for i in range(3):
    for n in range(3):
        part_c_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) >= 0, "Demand Lower Bound")

# Price Constraint
for n in range(2):  
    part_c_model.addConstr(p[0, n] <= p[0, n + 1], f"Price Constraint {i}")
    part_c_model.addConstr(p[1, n] <= p[1, n + 1], f"Price Constraint {i}")
    part_c_model.addConstr(p[2, n] <= p[2, n + 1], f"Price Constraint {i}")

for i in range(3): 
    for n in range(3): 
        part_c_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) <= product_capacity[i,n], "Max Demand")

# Solve our model
part_c_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 9 columns and 30 nonzeros
Model fingerprint: 0x7c0c1e87
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl    

In [47]:
# Value of the objective function
print("Revenue: ", round(part_c_model.objVal,2))

Revenue:  718382097.67


In [48]:
# Print the decision variables
print(part_c_model.printAttr('X'))


    Variable            X 
-------------------------
  Price[0,0]      383.848 
  Price[0,1]       2296.5 
  Price[0,2]      2351.88 
  Price[1,0]       2050.3 
  Price[1,1]      4160.71 
  Price[1,2]      6813.66 
  Price[2,0]      5870.93 
  Price[2,1]      5870.93 
  Price[2,2]      8558.94 
None


### (d)

In [49]:
# Create the optimization model
part_d_model = gb.Model("Question 1d): TechEssentials Certain Product Line")

p = part_d_model.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name="Price")

#Objective Function
part_d_model.setObjective(gb.quicksum(p[i,n]*(intercept_values[i][n] + slope[i][n]*p[i,n]) for i in range(3) for n in range(3)), GRB.MAXIMIZE)

for i in range(3):
    for n in range(3):
        part_d_model.addConstr((intercept_values[i][n] + slope[i][n]*p[i,n]) >= 0, "Demand Lower Bound")

# Price Constraint
for n in range(2):  
    part_d_model.addConstr(p[0, n] <= p[0, n + 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[1, n] <= p[1, n + 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[2, n] <= p[2, n + 1], f"Price Constraint {i}")

for n in range(2):  
    part_d_model.addConstr(p[n, 0] <= p[n + 1, 0], f"Price Constraint {i}")
    part_d_model.addConstr(p[n, 1] <= p[n + 1, 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[n, 2] <= p[n + 1, 2], f"Price Constraint {i}")

for i in range(3): 
    for n in range(3): 
        part_d_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) <= product_capacity[i,n], "Max Demand")

# Solve our model
part_d_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 30 rows, 9 columns and 42 nonzeros
Model fingerprint: 0x4ea948ad
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl   

In [50]:
# Value of the objective function
print("Revenue: ", round(part_d_model.objVal,2))

Revenue:  718382097.67


In [51]:
# Print the decision variables
print(part_d_model.printAttr('X'))


    Variable            X 
-------------------------
  Price[0,0]      383.848 
  Price[0,1]       2296.5 
  Price[0,2]      2351.88 
  Price[1,0]       2050.3 
  Price[1,1]      4160.71 
  Price[1,2]      6813.66 
  Price[2,0]      5870.93 
  Price[2,1]      5870.93 
  Price[2,2]      8558.94 
None


In [52]:
# Extract values from the tupledict
values = [v for v in part_d_model.getAttr('x', p).values()]

# Reshape the values
price_values = np.array(values).reshape(3, -1)

price_values

array([[ 383.84827161, 2296.49891813, 2351.87766702],
       [2050.29879442, 4160.70785608, 6813.65650408],
       [5870.9328096 , 5870.9328096 , 8558.94236073]])

In [53]:
demand = [[intercept_values[i][n] + slope[i][n] * price_values[i][n] for n in range(3)] for i in range(3)]
demand

[[17617.272892756177, 18895.120416101, 17837.666608596355],
 [18520.690188945133, 18423.070192958312, 17913.51187327708],
 [25197.89694029585, 12505.211952096714, 19656.65851562613]]

# Question 2

In [54]:
import gurobipy as gp
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

In [55]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\BasketballPlayers.csv")

In [56]:
m = gp.Model("BasketballTeamSelection")

In [57]:
# Add decision variables
players = df.index.tolist()
player_vars = m.addVars(players, vtype=GRB.BINARY, name="Player")

In [58]:
# Guard and Forward/Center allocation constraints
guards = df[df['Position'].isin(['G', 'G/F'])].index
forward_centers = df[df['Position'].isin(['F', 'C', 'F/C'])].index
m.addConstr(player_vars.sum(guards) >= 0.3 * 21, "MinGuards")
m.addConstr(player_vars.sum(forward_centers) >= 0.4 * 21, "MinForwardCenters")

<gurobi.Constr *Awaiting Model Update*>

In [59]:
# Skill average constraints
skills = ['Ball Handling', 'Shooting', 'Rebounding', 'Defense', 'Athletic Ability', 'Toughness', 'Mental Acuity']

for skill in skills:
    total_skill_points = gp.quicksum(player_vars[i] * df.at[i, skill] for i in players)
    m.addConstr(total_skill_points >= 2.05 * 21, f"Skill_{skill}_Adjusted")

In [60]:
# Exclusive invitation constraints
group1 = list(range(20, 25))
group2 = list(range(72, 79))
group3 = list(range(105, 115))
group4 = list(range(45, 50))
group5 = list(range(65, 70))
m.addConstr(gp.quicksum(player_vars[i] for i in group1) * gp.quicksum(player_vars[i] for i in group2) == 0, "Exclusive1")
m.addConstr(gp.quicksum(player_vars[i] for i in group3) <= (gp.quicksum(player_vars[i] for i in group4) + gp.quicksum(player_vars[i] for i in group5)), "DependentInvitations")

<gurobi.Constr *Awaiting Model Update*>

In [61]:
# At least one player from each range constraint
for start in range(1, 141, 10):
    end = start + 9
    m.addConstr(gp.quicksum(player_vars[i] for i in range(start, end+1)) >= 1, f"AtLeastOne_{start}-{end}")

m.addConstr(player_vars.sum() == 21, "Exactly21Invitations")

m.setObjective(gp.quicksum(player_vars[i] * df.loc[i, skills].sum() for i in players), GRB.MAXIMIZE)

In [62]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 25 rows, 150 columns and 1510 nonzeros
Model fingerprint: 0x7b25a2fb
Model has 1 quadratic constraint
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve added 35 rows and 0 columns
Presolve time: 0.00s
Presolved: 60 rows, 150 columns, 1241 nonzeros
Variable types: 0 continuous, 150 integer (150 binary)

Root relaxation: objective 3.620000e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

In [63]:
# Extract the solution
selected_players = [i for i in players if player_vars[i].X > 0.5]
print("Selected Players:", selected_players)

len(selected_players)

optimal_value = m.objVal

print(f"Optimal Objective Function Value: {optimal_value}")

num_guards_invited = sum(player_vars[i].X for i in guards)
print(f"Number of Guards Invited: {num_guards_invited}")

Selected Players: [4, 6, 15, 25, 36, 46, 55, 69, 73, 75, 89, 94, 103, 109, 110, 117, 130, 131, 132, 133, 140]
Optimal Objective Function Value: 362.0
Number of Guards Invited: 10.0


### (h)

In [65]:
import pandas as pd
from gurobipy import Model, GRB


print("\n####################################### PART H: FINDING MINIMUM INVITATIONS POSSIBLE #######################################################################\n")

# Read the player data
players_df = pd.read_csv('https://raw.githubusercontent.com/mredshaw/MODELS-APP/main/Assignment%202/BasketballPlayers.csv')

players_df['Average'] = players_df.iloc[:, 2:].mean(axis=1)

# Filter players with average skill above 2.05 (without resetting the index)
filtered_players_df = players_df[players_df['Average'] > 2.05]
filtered_players_df = filtered_players_df.drop(columns=['Average'])

# Number of players
num_players = len(filtered_players_df)

# Create the optimization model
model = Model("TrainingCampSelection")

# Create binary decision variables for each player (using original indices)
x = model.addVars(filtered_players_df.index, vtype=GRB.BINARY, name="Player")

# Pre-compute the player positions using original indices
guards = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['G', 'G/F']]
forwards_centers = [i for i in filtered_players_df.index if filtered_players_df.loc[i, 'Position'] in ['F', 'C', 'F/C']]

# Total number of players selected
total_players_selected = sum(x[i] for i in filtered_players_df.index)

# At least 30% of the invitations should go to guards
model.addConstr(sum(x[i] for i in guards) >= 0.3 * total_players_selected, "Min_30_percent_guards")

# At least 40% of the invitations should go to forwards/centers
model.addConstr(sum(x[i] for i in forwards_centers) >= 0.4 * total_players_selected, "Min_40_percent_forwards_centers")

# Limit the total number of invitations to 21
#model.addConstr(total_players_selected <= 21, "Total_Invitations_Limit")

# If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be, and vice versa
model.addConstr(sum(x[i] for i in filtered_players_df.index if 20 <= filtered_players_df.loc[i, 'Number'] <= 24) + sum(x[j] for j in filtered_players_df.index if 72 <= filtered_players_df.loc[j, 'Number'] <= 78) <= 1, "Group_20_24_vs_72_78")


# If any player from 105-114 (inclusive) is invited, at least one player from 45-49 (inclusive) and 65-69 (inclusive) must be invited
for i in [idx for idx in filtered_players_df.index if 105 <= filtered_players_df.loc[idx, 'Number'] <= 114]:
    model.addConstr(x[i] <= sum(x[j] for j in filtered_players_df.index if 45 <= filtered_players_df.loc[j, 'Number'] <= 49) + sum(x[k] for k in filtered_players_df.index if 65 <= filtered_players_df.loc[k, 'Number'] <= 69), f"Group_105_114_requires_{i}")


# At least one player must be invited from: 1-10, 11-20, 21-30, ..., 131-140, 141-150
for i in range(1, 151, 10):
    model.addConstr(sum(x[j] for j in filtered_players_df.index if i <= filtered_players_df.loc[j, 'Number'] < i + 10) >= 1, f"Group_{i}_{i+9}")


# Update the model
model.update()

# Print the number of constraints in the model
print("Number of constraints after adding:", len(model.getConstrs()))

# Check if there are any constraints in the model
constraints = model.getConstrs()
if not constraints:
    raise ValueError("No constraints found in the model")
last_feasible_solution = constraints[0]

# Change the objective function to minimize the total number of players selected
model.setObjective(total_players_selected, GRB.MINIMIZE)

# Find the smallest number of players that can be selected without causing infeasibility
min_players_selected = num_players
infeasible_constraint = None

for i in range(num_players, 0, -1):
    model.update()
    model.optimize()

    # Find the selected players
    selected_players = [i for i in filtered_players_df.index if x[i].X > 0.5]

# Print the details of the selected players
    print("Selected players:")
    for player in selected_players:
        player_data = filtered_players_df.loc[player]
        print(f"Player Number: {player_data['Number']}, Position: {player_data['Position']}")


    if model.status == GRB.INFEASIBLE:
        infeasible_constraint = model.getConstrs()[model.getConstrs().index(last_feasible_solution) + 1].ConstrName
        break
    elif model.status == GRB.OPTIMAL:
        min_players_selected = i
        # Update last_feasible_solution only if it's not the last constraint
        if model.getConstrs().index(last_feasible_solution) + 1 < len(model.getConstrs()):
            last_feasible_solution = model.getConstrs()[model.getConstrs().index(last_feasible_solution) + 1]


print(f"Value of the objective function (total number of players selected): {model.ObjVal}")
if infeasible_constraint:
    print(f"The constraint that caused infeasibility: {infeasible_constraint}")

print("\n####################################### FINDING WHAT CONSTRAINTS WOULD CAUSE INFEASIBILITY #######################################################################\n")

model.addConstr(total_players_selected <= model.ObjVal - 1, "Reduced_Player_Selection")

model.optimize()


if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Identifying the problematic constraint(s)...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Constraint causing infeasibility: {c.ConstrName}")


####################################### PART H: FINDING MINIMUM INVITATIONS POSSIBLE #######################################################################

Number of constraints after adding: 23
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 23 rows, 65 columns and 225 nonzeros
Model fingerprint: 0xc2c165cb
Variable types: 0 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 1 rows and 28 columns
Presolve time: 0.00s
Presolved: 22 rows, 37 columns, 135 nonzeros
Variable types: 0 continuous, 37 integer (23 binary)

Root relaxation: cutoff, 14 iterations, 0.00 seco